In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
# Gives a random binary vector
def get_b(n, p=0.5):
    return np.array( [np.random.choice([complex("0"),complex("1")], p=[1-p, p]) for i in range(n)] )

# In place, randomly selects an entry of 1 in b and replaces it with 1j.
def inject_tracker(b):
    support = [i for i in range(b.shape[0]) if b[i,] ]
    node = np.random.choice(support)
    b[node,] = complex("j")

# The collision function. The input is a vector y = Ax + b
def f(y):
    # We multiply each entry of y by the indicator that there is only one message at the corresponding node 
    return np.array( [ y[i,]*(y[i,].real + y[i,].imag == 1) for i in range(y.shape[0]) ] )

In [5]:
# Simulator
# A: Adjacency matrix an a square np array
# p: probability of injection at a given node
# num_messages: the number of messages to be tracked before ending the simulation
def bran(A, p, num_messages):
    n = A.shape[0]

    # Initialize external parameters
    survival_times = dict()
    alpha = 0
    need_injection = True

    # Initialize dynamical parameters (t=0)
    x = get_b(n, p)

    # This is the main loop
    while alpha < num_messages:
        # Get b_t
        b = get_b(n)

        # If it is possible and necessary to inject a tracker, do so, and update values accordingly
        if (np.sum(b) > 0) and need_injection:
            inject_tracker(b)
            need_injection = False
            alpha += 1
            survival_times[alpha] = 1

        # Calculate the next state
        x = f(np.matmul(A,x) + b)

        # Check if message alpha has survived, and if so, update accordingly
        if complex("j") in x:
            survival_times[alpha] += 1
        else:
            need_injection = True
    
    return survival_times

In [6]:
mean_survival_times = np.zeros((21,))
mean_survival_times[0,]=1

In [8]:
n = 100
for W in range(1,21):
    w = W/20
    A = np.zeros((n,n), dtype=int)
    for i in range(n):
        for j in range(n):
            A[i,j] = np.random.choice([0,1], p=[1-w,w])

    survival_times = bran(A, p=0.05, num_messages=5000)
    survival_times = [survival_times[alpha] for alpha in survival_times.keys()]
    mean_survival_times[W,] = np.mean(survival_times)

KeyboardInterrupt: 